In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

C:\Users\HARSHIT JAIN\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv(r'C:\Users\HARSHIT JAIN\Desktop\data science projects\Real_Estate_Data_science_Project\DATASETS\gurgaon_properties_missing_value_imputation\gurgaon_properties_missing_value_imputation.csv')

In [3]:
df.head()

,property_type,society,sector,price,price_per_sqrt,bedRoom,bathroom,balcony,floorNum,agePossession,built_up_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_score
0,falt,dlf park place,sector 54,5.75,24689.0,3,4,3,12.0,Relatively New,2329.0,0,1,0,0,0,2,174
1,falt,palam vihar society,sector 2,3.35,7444.0,4,4,3+,2.0,New Property,5000.0,1,0,0,0,0,1,66
2,falt,tulip violet,sector 69,1.40,8856.0,3,3,2,9.0,Relatively New,1428.0,0,0,0,1,0,1,120
3,falt,m3m golf hills phase 1,sector 79,1.70,10828.0,3,2,3,10.0,Relatively New,1570.0,0,0,0,0,0,1,0
4,falt,godrej nature plus,sector 33,1.35,8670.0,3,2,3+,17.0,New Property,1409.0,0,0,0,0,0,1,38


In [4]:
train_df = df.drop(columns=['society','price_per_sqrt'])

In [5]:
train_df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,floorNum,agePossession,built_up_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_score
0,falt,sector 54,5.75,3,4,3,12.0,Relatively New,2329.0,0,1,0,0,0,2,174
1,falt,sector 2,3.35,4,4,3+,2.0,New Property,5000.0,1,0,0,0,0,1,66
2,falt,sector 69,1.40,3,3,2,9.0,Relatively New,1428.0,0,0,0,1,0,1,120
3,falt,sector 79,1.70,3,2,3,10.0,Relatively New,1570.0,0,0,0,0,0,1,0
4,falt,sector 33,1.35,3,2,3+,17.0,New Property,1409.0,0,0,0,0,0,1,38


In [6]:
def categorize_luxury(score):
    if 0 <= score < 50 :
        return 'low'
    elif 50 <= score < 150:
        return 'medium'
    elif 150 <= score <=175 :
        return 'high'
    else :
        return None 

In [7]:
train_df['luxury_category'] = train_df['luxury_score'].apply(categorize_luxury)

In [8]:
train_df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,floorNum,agePossession,built_up_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_score,luxury_category
0,falt,sector 54,5.75,3,4,3,12.0,Relatively New,2329.0,0,1,0,0,0,2,174,high
1,falt,sector 2,3.35,4,4,3+,2.0,New Property,5000.0,1,0,0,0,0,1,66,medium
2,falt,sector 69,1.40,3,3,2,9.0,Relatively New,1428.0,0,0,0,1,0,1,120,medium
3,falt,sector 79,1.70,3,2,3,10.0,Relatively New,1570.0,0,0,0,0,0,1,0,low
4,falt,sector 33,1.35,3,2,3+,17.0,New Property,1409.0,0,0,0,0,0,1,38,low


In [10]:
def categorize_floor(floor):
    if 1 <= floor <= 2 :
        return 'low floor'
    elif 3 <= floor <= 10:
        return 'medium floor'
    elif 11 <= floor <=51 :
        return 'high floor'
    else :
        return None 

In [11]:
train_df['floor_category'] = train_df['floorNum'].apply(categorize_floor)

In [12]:
train_df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,floorNum,agePossession,built_up_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_score,luxury_category,floor_category
0,falt,sector 54,5.75,3,4,3,12.0,Relatively New,2329.0,0,1,0,0,0,2,174,high,high floor
1,falt,sector 2,3.35,4,4,3+,2.0,New Property,5000.0,1,0,0,0,0,1,66,medium,low floor
2,falt,sector 69,1.40,3,3,2,9.0,Relatively New,1428.0,0,0,0,1,0,1,120,medium,medium floor
3,falt,sector 79,1.70,3,2,3,10.0,Relatively New,1570.0,0,0,0,0,0,1,0,low,medium floor
4,falt,sector 33,1.35,3,2,3+,17.0,New Property,1409.0,0,0,0,0,0,1,38,low,high floor


In [13]:
train_df.drop(columns=['luxury_score','floorNum'],inplace=True)

In [14]:
train_df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_category,floor_category
0,falt,sector 54,5.75,3,4,3,Relatively New,2329.0,0,1,0,0,0,2,high,high floor
1,falt,sector 2,3.35,4,4,3+,New Property,5000.0,1,0,0,0,0,1,medium,low floor
2,falt,sector 69,1.40,3,3,2,Relatively New,1428.0,0,0,0,1,0,1,medium,medium floor
3,falt,sector 79,1.70,3,2,3,Relatively New,1570.0,0,0,0,0,0,1,low,medium floor
4,falt,sector 33,1.35,3,2,3+,New Property,1409.0,0,0,0,0,0,1,low,high floor


In [15]:
train_df.drop(columns=['pooja room', 'study room', 'others'])

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,falt,sector 54,5.75,3,4,3,Relatively New,2329.0,1,0,2,high,high floor
1,falt,sector 2,3.35,4,4,3+,New Property,5000.0,0,0,1,medium,low floor
2,falt,sector 69,1.40,3,3,2,Relatively New,1428.0,0,0,1,medium,medium floor
3,falt,sector 79,1.70,3,2,3,Relatively New,1570.0,0,0,1,low,medium floor
4,falt,sector 33,1.35,3,2,3+,New Property,1409.0,0,0,1,low,high floor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,falt,sector 83,1.42,3,3,2,Relatively New,1972.0,0,0,0,medium,low floor
3551,falt,sector 56,0.65,1,1,1,Moderately Old,839.0,0,0,1,medium,None
3552,house,sector 6,0.40,4,2,2,Moderately Old,1600.0,0,0,1,low,low floor
3553,falt,sector 89,1.28,3,3,2,New Property,1611.0,0,0,1,medium,medium floor


In [16]:
train_df.to_csv('gurgaon_properties_post_feature_selection_v2.csv', index=False)